In [ ]:
import json
from openai import OpenAI

client = OpenAI(
    api_key="sk-", 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

class CompressedAgent:
    def __init__(self, model="qwen-plus"):
        self.model = model
        self.short_term_memory = []  # L1: 存储最近 N 轮原始对话
        self.long_term_summary = ""   # L2: 存储之前的压缩背景
        self.max_short_term_rounds = 3 # 设定阈值：超过3轮就压缩一次

    def _summarize_memory(self, old_summary, new_exchanges):
        """
        核心：上下文压缩器 (Context Compressor)
        利用 LLM 将 旧摘要 + 新对话 融合成 新摘要
        """
        print("\n--- 🔄 正在触发 Context Engineering: 压缩记忆中... ---")
        
        # 构造一个专门用于总结的 Prompt
        summarize_prompt = f"""
        你是一个记忆管理专家。请将以下对话内容浓缩并更新到现有的“背景摘要”中。
        要求：保留关键事实（如姓名、数额、决定）、当前进度和用户的偏好。
        
        现有的背景摘要: {old_summary}
        
        新增的对话内容:
        {new_exchanges}
        
        请生成一个新的、精炼的背景摘要：
        """
        
        response = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": summarize_prompt}]
        )
        return response.choices[0].message.content

    def chat(self, user_input):
        # 1. 构建当前的最终 Prompt
        # 我们把 Long-term Summary 作为 System 内容的一部分灌入
        full_system_prompt = f"""你是一个智能助手。
        以下是之前的对话背景摘要（长期记忆）：
        {self.long_term_summary if self.long_term_summary else "目前尚无历史背景。"}
        """
        
        # 组合消息：System(带摘要) + Short-term Memory + 当前用户输入
        current_messages = [{"role": "system", "content": full_system_prompt}]
        current_messages.extend(self.short_term_memory)
        current_messages.append({"role": "user", "content": user_input})

        # 2. 调用模型
        response = client.chat.completions.create(
            model=self.model,
            messages=current_messages
        )
        
        assistant_reply = response.choices[0].message.content
        
        # 3. 更新短期记忆
        self.short_term_memory.append({"role": "user", "content": user_input})
        self.short_term_memory.append({"role": "assistant", "content": assistant_reply})

        # 4. 检查是否需要压缩
        # 如果短期记忆超过了设定轮数 (每轮含 user 和 assistant，所以 *2)
        if len(self.short_term_memory) > self.max_short_term_rounds * 2:
            # 提出最老的一轮（2条消息）进行压缩
            to_compress = self.short_term_memory[:2]
            self.short_term_memory = self.short_term_memory[2:] # 剩下的作为短期记忆
            
            exchanges_str = f"User: {to_compress[0]['content']}\nAssistant: {to_compress[1]['content']}"
            
            # 更新长期摘要
            self.long_term_summary = self._summarize_memory(self.long_term_summary, exchanges_str)

        return assistant_reply

# --- 测试运行 ---
agent = CompressedAgent()

print("Round 1:", agent.chat("你好，我叫张三，我是一名AI工程师。"))
print("Round 2:", agent.chat("我最近在研究大模型的Agent技术。"))
print("Round 3:", agent.chat("我想做一个能帮我写代码的工具。"))
# 此时会触发压缩...
print("Round 4:", agent.chat("你还记得我叫什么名字，以及我在研究什么吗？"))

Round 1: 你好，张三！很高兴认识你。作为一名AI工程师，你一定在机器学习、自然语言处理或相关领域有不少经验吧？如果有关于AI技术、项目实践或者行业趋势等方面的话题，我很乐意和你交流探讨！😊
Round 2: 哇，Agent技术可是当前大模型应用中最热门的方向之一了！👏

你研究的是哪方面的 Agent 呢？比如：

- **任务型 Agent**（如能自动完成订票、写代码、数据分析等复杂任务）
- **多智能体系统**（多个 Agent 协作或博弈）
- **记忆与规划机制**（如 ReAct、Reflexion、Chain-of-Thought + Planning）
- **工具使用能力**（Tool Use / Function Calling）
- **自主目标设定**（像 AutoGPT 那样的自我驱动系统）

最近像 **Meta 的 ToolLLM**、**Google 的 PaLM-E**、**Microsoft 的 TaskMatrix.AI**，还有 **LangChain + LLMs 构建的自主 Agent** 都挺火的。你在实践中有没有用到什么框架或者遇到哪些挑战？比如稳定性、幻觉控制、长期记忆管理之类的？

我也很乐意分享一些前沿进展或帮你一起思考设计思路 😊
Round 3: 太棒了！一个能帮你写代码的 AI Agent，正是当前“程序员生产力革命”的核心方向之一。像 GitHub Copilot、CodeLlama、Devin（by Cursor）这类工具已经展示了巨大潜力。我们可以一起设计一个更智能、更贴合你个人习惯的**个性化代码助手 Agent**。

下面我帮你梳理一下这个工具的关键模块和实现思路：

---

### 🎯 1. 核心功能目标（你可以根据需要选择）
- ✅ **代码补全增强版**：不只是单行补全，而是理解上下文后生成函数/类/模块。
- ✅ **需求→代码**：输入自然语言需求（如“写一个 FastAPI 接口，接收 JSON 并存入 PostgreSQL”），自动生成可运行代码。
- ✅ **Bug 诊断与修复**：分析报错日志或异常行为，定位问题并建议修复。
- ✅ **代码重构建议**：识别坏味道（code smells）、优化结构、提升性能。
- ✅ **单元测试生成**：为已有函数自动生成 p